In [1]:
#adapted from @bact at https://colab.research.google.com/drive/1hdtmwTXHLrqNmDhDqHnTQGpDVy1aJc4t
import json
import pandas as pd
import numpy as np
import re
import pycrfsuite
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from pythainlp.tokenize import word_tokenize
from pythainlp.tag import pos_tag
with open('data/talks.transcript.th-en.24-10-2019_11:12.json', 'r') as f:
    data = json.load(f)

In [2]:
talk_idx = []
idx = []
th_phrases = []
en_phrases = []

for i in range(len(data)):
    nb_phrases = min(len(data[i]['en']),len(data[i]['th']))
    for j in range(nb_phrases):
        talk_idx.append(i)
        th_phrases.append(data[i]['th'][j])
        en_phrases.append(data[i]['en'][j])
        if data[i]['en'][j][-1]=='.':
            idx.append(1)
        else:
            idx.append(0)

In [3]:
phrase_df = pd.DataFrame({'talk_idx':talk_idx,'en_phrase':en_phrases ,'th_phrase':th_phrases, 'idx':idx})
phrase_df.idx.sum()

136463

In [4]:
all_sentences = []
for i in range(phrase_df.talk_idx.max()):
    df = phrase_df[phrase_df.talk_idx==i]
    sentences = []
    for j,row in df.iterrows():
        sentences.append(row.th_phrase)
        if row.idx==1:
            sentences.append(' |')
    joined_sentences = ''.join(sentences)
    #remove parantheses like (audience claps)
    joined_sentences = re.sub(r'\([^)]*\)', '', joined_sentences)
    #skip if talk is less than 2 characters
    if len(joined_sentences)<2: continue
    #remove | at the last sentence if present
    if joined_sentences[-1]=='|': joined_sentences = joined_sentences[:-1]
    all_sentences.append(joined_sentences)

In [5]:
len(all_sentences)

1543

In [6]:
all_sentences[0]

'บนหลังอาชาแห้งกร่องดอนกิโฆเต้ พระเอกของเราบุกตะลุยสู้กับกองทัพยักษ์ |ในสายตาของเขา มันเป็นหน้าที่ของเขาที่จะปราบอสูรร้ายเหล่านี้ในนามแห่งหญิงอันเป็นที่รักของเขา ดุลสิเนอา |ทว่า การกระทำอันหาญกล้านี้ก็สูญเปล่า |เมื่อซานโซ่ ปันซ่า ผู้รับใช้ของเขาอธิบายครั้งแล้วครั้งเล่า ว่าสิ่งเหล่านี้จะเป็นยักษ์ก็หาไม่พวกมันเป็นเพียงกังหันลมเท่านั้น |ดอนกิโฆเต้ หาได้เสียความแน่วแน่แทงทวนของเขาเข้าไปยังใบพัดอย่างจัง |ด้วยพลังใจที่ไม่เคยถดถอยอัศวินผู้นั้นยืนขึ้นอย่างภาคภูมิและยิ่งเชื่อมั่นในปฏิบัติการของเขามากขึ้น |ลำดับเหตุการณ์นี้ครอบคลุมเรื่องราวส่วนใหญ่ของดอนกิโฆเต้ ที่เป็นที่รักมหากาพย์ ไร้ตรรกะ และมีชีวิตชีวาของ อลองโซ กีฆานาผู้กลายเป็น ดอนกิโฆเต้ แห่งลามันช่าผู้ซุ่มซ่ามแต่กล้าหาญหรือที่รู้จักกันในนาม ขุนนางต่ำศักดิ์นักฝัน |แต่เดิมวรรณกรรมนี้มีสองเล่มบรรยายเรื่องราวของดอนกิโฆเต้ในขณะที่เขาเดินทางผ่านตอนกลาง และตอนเหนือของสเปนเพื่อต่อสู้กับบรรดาปิศาจร้าย |แม้จินตนาการในเรื่อง ดอนกิโฆเต้ จะสูงล้ำเหนือเมฆผู้ประพันธ์ มิเกล เด เซร์บันเตสก็ไม่เคยนึกฝันว่าหนังสือของเขาจะกลายเป็นนิยายที่ขายดีที่สุดตลอดกาล 

In [7]:
all_tuples = []
for i in range(len(all_sentences)):
    tuples = []
    for s in all_sentences[i].split('|'):
        s_lst = word_tokenize(s)
        for j in range(len(s_lst)):
            lab = 'E' if j==len(s_lst)-1 else 'I'
            tuples.append((s_lst[j],lab))
    all_tuples.append(tuples)

In [8]:
len(all_tuples)

1543

In [9]:
word_tokenize('สวัสดีนะคะ')

['สวัสดี', 'นะคะ']

In [10]:
enders = ["ครับ","ค่ะ","คะ","นะคะ","นะ","จ้ะ","จ้า","จ๋า","ฮะ", #ending honorifics
          #enders
          "ๆ","ได้","แล้ว","ด้วย","เลย","มาก","น้อย","กัน","เช่นกัน","เท่านั้น",
          "อยู่","ลง","ขึ้น","มา","ไป","ไว้","เอง","อีก","ใหม่","จริงๆ",
          "บ้าง","หมด","ทีเดียว","เดียว",
          #demonstratives
          "นั้น","นี้","เหล่านี้","เหล่านั้น",
          #questions
          "อย่างไร","ยังไง","หรือไม่","มั้ย","ไหน","อะไร","ทำไม","เมื่อไหร่"]
starters = ["ผม","ฉัน","ดิฉัน","ชั้น","คุณ","มัน","เขา","เค้า",
            "เธอ","เรา","พวกเรา","พวกเขา", #pronouns
            #connectors
            "และ","หรือ","แต่","เมื่อ","ถ้า","ใน",
            "ด้วย","เพราะ","เนื่องจาก","ซึ่ง","ไม่",
            "ตอนนี้","ทีนี้","ดังนั้น","เพราะฉะนั้น","ฉะนั้น",
            "ตั้งแต่","ในที่สุด",
            #demonstratives
            "นั้น","นี้","เหล่านี้","เหล่านั้น"]

def extract_features(doc, window=2, max_n_gram=3):
    doc_features = []
    #paddings for word and POS
#     doc_pos = ['xxpad' for i in range(window)] + \
#         [p for (w,p) in pos_tag(doc,engine='artagger', corpus='orchid')] + ['xxpad' for i in range(window)]
    doc = ['xxpad' for i in range(window)] + doc + ['xxpad' for i in range(window)]
    doc_ender = []
    doc_starter = []
    #add enders
    for i in range(len(doc)):
        if doc[i] in enders:
            doc_ender.append('ender')
        else:
            doc_ender.append('normal')
    #add starters
    for i in range(len(doc)):
        if doc[i] in starters:
            doc_starter.append('starter')
        else:
            doc_starter.append('normal')
    #for each word
    for i in range(window, len(doc)-window):
        #bias term
        word_features = ['bias'] 
        
        #ngram features
        for n_gram in range(1, min(max_n_gram+1,2+window*2)):
            for j in range(i-window,i+window+2-n_gram):
                feature_position = f'{n_gram}_{j-i}_{j-i+n_gram}'
                word_ = f'{"|".join(doc[j:(j+n_gram)])}'
                word_features += [f'word_{feature_position}={word_}']
#                 pos_ =f'{"|".join(doc_pos[j:(j+n_gram)])}'
#                 word_features += [f'pos_{feature_position}={pos_}']
                ender_ =  f'{"|".join(doc_ender[j:(j+n_gram)])}'
                word_features += [f'ender_{feature_position}={ender_}']
                starter_ =  f'{"|".join(doc_starter[j:(j+n_gram)])}'
                word_features += [f'starter_{feature_position}={starter_}']
        
#         #number of verbs to the left and right
#         nb_verbs_left = 0
#         for l in range(i)[::-1]:
#             if doc[l]=='<space>': break
#             if doc_pos[l]=='VACT': nb_verbs_left+=1
#         nb_verbs_right = 0
#         for r in range(i+1,len(doc)):
#             if doc[r]=='<space>': break
#             if doc_pos[r]=='VACT': nb_verbs_right+=1
#         word_features += [f'nb_verbs_left={nb_verbs_left}',f'nb_verbs_right={nb_verbs_right}']
        
        #append to feature per word
        doc_features.append(word_features)
    return doc_features

In [11]:
extract_features(word_tokenize('ฉันชอบกินมะนาว ฉันชอบกินส้ม แต่ดี'), window = 2, max_n_gram = 3)

[['bias',
  'word_1_-2_-1=xxpad',
  'ender_1_-2_-1=normal',
  'starter_1_-2_-1=normal',
  'word_1_-1_0=xxpad',
  'ender_1_-1_0=normal',
  'starter_1_-1_0=normal',
  'word_1_0_1=ฉัน',
  'ender_1_0_1=normal',
  'starter_1_0_1=starter',
  'word_1_1_2=ชอบ',
  'ender_1_1_2=normal',
  'starter_1_1_2=normal',
  'word_1_2_3=กิน',
  'ender_1_2_3=normal',
  'starter_1_2_3=normal',
  'word_2_-2_0=xxpad|xxpad',
  'ender_2_-2_0=normal|normal',
  'starter_2_-2_0=normal|normal',
  'word_2_-1_1=xxpad|ฉัน',
  'ender_2_-1_1=normal|normal',
  'starter_2_-1_1=normal|starter',
  'word_2_0_2=ฉัน|ชอบ',
  'ender_2_0_2=normal|normal',
  'starter_2_0_2=starter|normal',
  'word_2_1_3=ชอบ|กิน',
  'ender_2_1_3=normal|normal',
  'starter_2_1_3=normal|normal',
  'word_3_-2_1=xxpad|xxpad|ฉัน',
  'ender_3_-2_1=normal|normal|normal',
  'starter_3_-2_1=normal|normal|starter',
  'word_3_-1_2=xxpad|ฉัน|ชอบ',
  'ender_3_-1_2=normal|normal|normal',
  'starter_3_-1_2=normal|starter|normal',
  'word_3_0_3=ฉัน|ชอบ|กิน',
  'end

In [12]:
#target
y = [[l for (w,l) in t] for t in all_tuples]
#features
x_pre = [[w for (w,l) in t] for t in all_tuples]
x = [extract_features(x_, window=2, max_n_gram = 3) for x_ in x_pre]

In [13]:
len(x),len(y)

(1543, 1543)

In [14]:
# Split train and test set at 80/20 proportion
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1412)

In [15]:
x_train[0][:5]

[['bias',
  'word_1_-2_-1=xxpad',
  'ender_1_-2_-1=normal',
  'starter_1_-2_-1=normal',
  'word_1_-1_0=xxpad',
  'ender_1_-1_0=normal',
  'starter_1_-1_0=normal',
  'word_1_0_1=วงดนตรี',
  'ender_1_0_1=normal',
  'starter_1_0_1=normal',
  'word_1_1_2=ที่',
  'ender_1_1_2=normal',
  'starter_1_1_2=normal',
  'word_1_2_3=คุณ',
  'ender_1_2_3=normal',
  'starter_1_2_3=starter',
  'word_2_-2_0=xxpad|xxpad',
  'ender_2_-2_0=normal|normal',
  'starter_2_-2_0=normal|normal',
  'word_2_-1_1=xxpad|วงดนตรี',
  'ender_2_-1_1=normal|normal',
  'starter_2_-1_1=normal|normal',
  'word_2_0_2=วงดนตรี|ที่',
  'ender_2_0_2=normal|normal',
  'starter_2_0_2=normal|normal',
  'word_2_1_3=ที่|คุณ',
  'ender_2_1_3=normal|normal',
  'starter_2_1_3=normal|starter',
  'word_3_-2_1=xxpad|xxpad|วงดนตรี',
  'ender_3_-2_1=normal|normal|normal',
  'starter_3_-2_1=normal|normal|normal',
  'word_3_-1_2=xxpad|วงดนตรี|ที่',
  'ender_3_-1_2=normal|normal|normal',
  'starter_3_-1_2=normal|normal|normal',
  'word_3_0_3=วงด

In [16]:
# Train model
trainer = pycrfsuite.Trainer(verbose=True)

for xseq, yseq in zip(x_train, y_train):
    trainer.append(xseq, yseq)

trainer.set_params({
    'c1': 1,
    'c2': 0,
    'max_iterations': 1000,
    'feature.possible_transitions': True,
})

trainer.train('sentenceseg-crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 7025657
Seconds required: 66.619

L-BFGS optimization
c1: 1.000000
c2: 0.000000
num_memories: 6
max_iterations: 1000
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 665590.334399
Feature norm: 1.000000
Error norm: 670709.750956
Active features: 921026
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 10.161

***** Iteration #2 *****
Loss: 611165.642994
Feature norm: 0.922394
Error norm: 647669.653883
Active features: 684793
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 4.842

***** Iteration #3 *****
Loss: 444874.635335
Feature norm: 0.504229
Error norm: 827083.792853
Active features: 201476
Line search trials: 3
Line search step: 0.250000
Second

***** Iteration #43 *****
Loss: 143932.789054
Feature norm: 22.388246
Error norm: 71302.228371
Active features: 148807
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 4.616

***** Iteration #44 *****
Loss: 141938.422648
Feature norm: 23.415785
Error norm: 16727.197064
Active features: 148932
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 5.247

***** Iteration #45 *****
Loss: 141389.558160
Feature norm: 23.936606
Error norm: 16175.954792
Active features: 148907
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 5.251

***** Iteration #46 *****
Loss: 140244.222740
Feature norm: 25.273108
Error norm: 28212.598837
Active features: 147168
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 4.596

***** Iteration #47 *****
Loss: 138657.268872
Feature norm: 28.844406
Error norm: 62180.493272
Active features: 143907
Line search trials: 1
Line search

***** Iteration #84 *****
Loss: 118417.478235
Feature norm: 93.524439
Error norm: 1238.553798
Active features: 75439
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 5.811

***** Iteration #85 *****
Loss: 118372.802103
Feature norm: 94.003143
Error norm: 2055.900442
Active features: 75197
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 5.332

***** Iteration #86 *****
Loss: 118330.039943
Feature norm: 94.501492
Error norm: 1884.762754
Active features: 74877
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 4.532

***** Iteration #87 *****
Loss: 118295.030604
Feature norm: 94.936634
Error norm: 2145.704311
Active features: 74629
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 4.882

***** Iteration #88 *****
Loss: 118263.807014
Feature norm: 95.327934
Error norm: 730.883722
Active features: 74184
Line search trials: 1
Line search step: 1.00

***** Iteration #131 *****
Loss: 117948.632043
Feature norm: 99.543657
Error norm: 353.826841
Active features: 68104
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 5.153

***** Iteration #132 *****
Loss: 117947.143490
Feature norm: 99.564138
Error norm: 566.728067
Active features: 68061
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 5.151

***** Iteration #133 *****
Loss: 117945.653405
Feature norm: 99.586039
Error norm: 684.719545
Active features: 67998
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 8.304

***** Iteration #134 *****
Loss: 117944.272566
Feature norm: 99.602702
Error norm: 328.920087
Active features: 67947
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.372

***** Iteration #135 *****
Loss: 117943.013464
Feature norm: 99.628795
Error norm: 686.347902
Active features: 67892
Line search trials: 1
Line search step: 1.0

***** Iteration #173 *****
Loss: 117913.824283
Feature norm: 100.191976
Error norm: 305.393405
Active features: 67164
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 6.214

***** Iteration #174 *****
Loss: 117913.415817
Feature norm: 100.193614
Error norm: 427.060827
Active features: 67143
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 6.529

***** Iteration #175 *****
Loss: 117912.963562
Feature norm: 100.200703
Error norm: 521.699813
Active features: 67133
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 6.412

***** Iteration #176 *****
Loss: 117912.638283
Feature norm: 100.205070
Error norm: 733.597579
Active features: 67119
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 6.517

***** Iteration #177 *****
Loss: 117912.037935
Feature norm: 100.215756
Error norm: 688.538752
Active features: 67114
Line search trials: 1
Line search step

***** Iteration #216 *****
Loss: 117898.573767
Feature norm: 100.462730
Error norm: 529.337040
Active features: 66795
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 6.092

***** Iteration #217 *****
Loss: 117898.231485
Feature norm: 100.464659
Error norm: 297.356823
Active features: 66799
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 6.798

***** Iteration #218 *****
Loss: 117898.029283
Feature norm: 100.471397
Error norm: 415.245624
Active features: 66789
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 6.237

***** Iteration #219 *****
Loss: 117897.810926
Feature norm: 100.474182
Error norm: 334.752646
Active features: 66797
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 6.281

***** Iteration #220 *****
Loss: 117897.684487
Feature norm: 100.479625
Error norm: 383.868828
Active features: 66778
Line search trials: 1
Line search step

***** Iteration #255 *****
Loss: 117890.942470
Feature norm: 100.582511
Error norm: 594.565365
Active features: 66670
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 5.966

***** Iteration #256 *****
Loss: 117890.715923
Feature norm: 100.583489
Error norm: 350.657264
Active features: 66673
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 6.107

***** Iteration #257 *****
Loss: 117890.596862
Feature norm: 100.586842
Error norm: 554.344990
Active features: 66658
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 6.193

***** Iteration #258 *****
Loss: 117890.440222
Feature norm: 100.587854
Error norm: 381.271034
Active features: 66643
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 6.410

***** Iteration #259 *****
Loss: 117890.274659
Feature norm: 100.592080
Error norm: 539.669024
Active features: 66643
Line search trials: 1
Line search step

***** Iteration #300 *****
Loss: 117884.321465
Feature norm: 100.663232
Error norm: 313.179266
Active features: 66418
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 5.642

***** Iteration #301 *****
Loss: 117884.171042
Feature norm: 100.663895
Error norm: 225.273322
Active features: 66422
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 10.311

***** Iteration #302 *****
Loss: 117884.155327
Feature norm: 100.664438
Error norm: 584.432939
Active features: 66422
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 5.295

***** Iteration #303 *****
Loss: 117883.951432
Feature norm: 100.665304
Error norm: 543.392376
Active features: 66423
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 5.240

***** Iteration #304 *****
Loss: 117883.785042
Feature norm: 100.665353
Error norm: 439.033084
Active features: 66413
Line search trials: 1
Line search ste

In [ ]:
# ***** Iteration #320 *****
# Loss: 117881.769524
# Feature norm: 100.673745
# Error norm: 195.321045
# Active features: 66365
# Line search trials: 2
# Line search step: 0.500000
# Seconds required for this iteration: 10.882

# L-BFGS terminated with the stopping criteria
# Total seconds required for training: 1886.993

# Storing the model
# Number of active features: 66365 (7025657)
# Number of active attributes: 38107 (6885229)
# Number of active labels: 2 (2)
# Writing labels
# Writing attributes
# Writing feature references for transitions
# Writing feature references for attributes
# Seconds required: 2.004


In [17]:
# Predict (using test set)
tagger = pycrfsuite.Tagger()
tagger.open('sentenceseg-crf.model')
y_pred = [tagger.tag(xseq) for xseq in x_test]

In [18]:
# Evaluate at word-level
labels = {'E': 0, "I": 1} # classification_report() needs values in 0s and 1s
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

print(classification_report(
    truths, predictions,
    target_names=["E", "I"]))

              precision    recall  f1-score   support

           E       0.74      0.70      0.72     28437
           I       0.99      0.99      0.99    664893

    accuracy                           0.98    693330
   macro avg       0.86      0.85      0.85    693330
weighted avg       0.98      0.98      0.98    693330



## `sentence_tokenize` Function

In [19]:
def sentence_tokenize(s):
    toks = word_tokenize(s)
    feat = extract_features(toks)
    labs = tagger.tag(feat)
    sentences = []
    sentence = ''
    for i, w in enumerate(toks):
        sentence = sentence + w
        if labs[i]=='E':
            sentences.append(sentence)
            sentence=''
    return sentences

In [20]:
s = 'เธอคือหุ่นยนต์รูปแบบใหม่ที่ฉันคนนี้สร้างขึ้นมาเธอมีความสามารถในการคิดและรู้สึกเหมือนมนุษย์ เธอสามารถตัดสินใจด้วยตัวเอง แต่ถึงอย่างนั้นมันก็เป็นดาบสองคม ที่อาจเป็นอันตรายถ้าเธอคิดว่ามนุษย์เป็นภัย จนแหกกฎข้อแรกที่หุ่นยนต์ห้ามทำร้ายมนุษย์ ฉันจำเป็นต้องใช้เวลาทดสอบระบบอีก 30 ปีเพื่อความมั่นใจว่าเธอจะไม่เป็นอันตรายกับมนุษย์ แต่ฉันคงจะมีชีวิตอยู่ไม่ถึงวันนั้น ดังนั้นฉันจึงปิดผนึกเธอเอาไว้ในแคปซูลจนกว่าระบบต่าง ๆ จะพร้อมเสียก่อน'

In [21]:
sentence_tokenize(s)

['เธอคือหุ่นยนต์รูปแบบใหม่ที่ฉันคนนี้สร้างขึ้นมาเธอมีความสามารถในการคิดและรู้สึกเหมือนมนุษย์ ',
 'เธอสามารถตัดสินใจด้วยตัวเอง ',
 'แต่ถึงอย่างนั้นมันก็เป็นดาบสองคม ที่อาจเป็นอันตรายถ้าเธอคิดว่ามนุษย์เป็นภัย จนแหกกฎข้อแรกที่หุ่นยนต์ห้ามทำร้ายมนุษย์ ',
 'ฉันจำเป็นต้องใช้เวลาทดสอบระบบอีก 30 ปีเพื่อความมั่นใจว่าเธอจะไม่เป็นอันตรายกับมนุษย์ ',
 'แต่ฉันคงจะมีชีวิตอยู่ไม่ถึงวันนั้น ',
 'ดังนั้นฉันจึงปิดผนึกเธอเอาไว้ในแคปซูลจนกว่าระบบต่าง ๆ จะพร้อมเสียก่อน']

## Error Analysis

In [22]:
results = []
for i in range(len(y_test)):
    s=0
    for j in range(len(y_test[i])):
        results.append({'sentence_idx':f'{str(i).zfill(3)}_{str(s).zfill(3)}',
                        'word':x_test[i][j][7].split('=')[1],
                        'y':y_test[i][j],
                        'pred':y_pred[i][j]})
        if y_test[i][j]=='E': s+=1
result_df = pd.DataFrame(results)[['sentence_idx','word','y','pred']]
result_df['wrong_flag'] = result_df.apply(lambda row: 0 if row.y==row.pred else 1,1)

In [23]:
result_df[result_df.sentence_idx=='000_000']

,sentence_idx,word,y,pred,wrong_flag
0,000_000,ชาร์ลี,I,I,0
1,000_000,,I,I,0
2,000_000,ออ,I,I,0
3,000_000,สบ,I,I,0
4,000_000,อน,I,I,0
5,000_000,,I,I,0
6,000_000,เริ่ม,I,I,0
7,000_000,สะอึก,I,I,0
8,000_000,ใน,I,I,0
9,000_000,ปี,I,I,0


In [24]:
#space correct
space_df = result_df.copy()
space_df = space_df[space_df.word==' ']
space_df.wrong_flag.mean(), space_df.shape

(0.18054657655010284, (85075, 5))

In [25]:
print(classification_report(
    space_df.y, space_df.pred,
    target_names=["E", "I"]))

              precision    recall  f1-score   support

           E       0.74      0.70      0.72     28089
           I       0.86      0.88      0.87     56986

    accuracy                           0.82     85075
   macro avg       0.80      0.79      0.79     85075
weighted avg       0.82      0.82      0.82     85075



In [26]:
#               precision    recall  f1-score   support

#            E       0.74      0.70      0.72     28089
#            I       0.86      0.88      0.87     56986

#     accuracy                           0.82     85075
#    macro avg       0.80      0.79      0.79     85075
# weighted avg       0.82      0.82      0.82     85075

In [27]:
#rule-based prediction
result_df['word_before'] = result_df.word.shift(1)
result_df['rule_pred'] = result_df.apply(lambda row: 'E' if row['word_before']=='ครับ' else row['pred'],1)
result_df['wrong_flag'] = result_df.apply(lambda row: 0 if row.y==row.rule_pred else 1,1)

In [28]:
#space correct
space_df = result_df.copy()
space_df = space_df[space_df.word==' ']
space_df.wrong_flag.mean(), space_df.shape

(0.18313253012048192, (85075, 7))

In [29]:
print(classification_report(
    space_df.y, space_df.rule_pred,
    target_names=["E", "I"]))

              precision    recall  f1-score   support

           E       0.73      0.71      0.72     28089
           I       0.86      0.87      0.86     56986

    accuracy                           0.82     85075
   macro avg       0.79      0.79      0.79     85075
weighted avg       0.82      0.82      0.82     85075



In [30]:
result_df[result_df.sentence_idx=='004_044']

,sentence_idx,word,y,pred,wrong_flag,word_before,rule_pred
9251,004_044,ไง,I,I,0,,I
9252,004_044,ครับ,I,I,0,ไง,I
9253,004_044,,I,I,1,ครับ,E
9254,004_044,ไม่,I,I,0,,I
9255,004_044,เห็น,I,I,0,ไม่,I
9256,004_044,ยาก,I,I,0,เห็น,I
9257,004_044,เลย,I,I,0,ยาก,I
9258,004_044,ใช่ไหม,I,I,0,เลย,I
9259,004_044,ครับ,I,I,0,ใช่ไหม,I
9260,004_044,,E,E,0,ครับ,E


In [42]:
pd.set_option('display.max_colwidth', 0)
result_agg = result_df.groupby('sentence_idx').agg({'word': lambda x: ''.join(x),
                                                    'wrong_flag': max,
                                                    'pred': lambda x: '|'.join(x),
                                                    'y': lambda x: '|'.join(x)}).reset_index()    
#clean up
result_agg['word'] = result_agg.word.map(lambda x: x.replace(' ',''))
result_agg = result_agg[result_agg.word!=''].reset_index(drop=True) 
#last word and first word
result_agg['last_word'] = result_agg.word.map(lambda x: word_tokenize(x)[-1])
result_agg['first_word'] = result_agg.word.map(lambda x: word_tokenize(x)[0]).shift(-1)

In [43]:
def see_sentences(sentence_idx):
    df = result_agg[result_agg.sentence_idx==sentence_idx]
    idx = df.index[0]
    if idx>0:
        print(f'Before: {result_agg.iloc[idx-1,1]}')
        print(f'Predicted: {result_agg.iloc[idx-1,3]}')
        print('\n')
    print(f'Current: {result_agg.iloc[idx,1]}')
    print(f'Predicted: {result_agg.iloc[idx,3]}')
    print('\n')
    if idx < result_agg.shape[0]-1:
        print(f'After: {result_agg.iloc[idx+1,1]}')
        print(f'Predicted: {result_agg.iloc[idx+1,3]}')

In [44]:
see_sentences('005_022')

Before: เราลืมความรับผิดชอบของมนุษย์ไปนั่นเป็นปัญหาข้อแรกปัญหาข้อที่สองคือเราลืมผู้คนอย่างพวกคุณไปเลยเราลืมเรื่องการยกระดับชีวิตของคนปกติ
Predicted: I|I|I|I|I|I|E|I|I|I|I|I|I|I|I|I|I|I|I|I|I|I|I|I|I|I|I|I|I|I|I|I|I


Current: เราลืมภารกิจในการทำให้คนที่ไม่ได้มีปัญหาเขามีความสุขมากขึ้น
Predicted: I|I|I|I|I|I|I|I|I|I|I|I|I|I|E


After: มีชิวิตที่เติมเต็มและเป็นประโยชน์มากขึ้นแล้วคำว่า"อัจฉริยะ""พรสวรรค์"กลายเป็นคำหยาบ
Predicted: I|I|I|I|I|I|I|I|I|I|E|I|I|I|I|I|I|I|I|I|I|I|I|I|E


In [45]:
#widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

opt = list(result_agg[result_agg.wrong_flag==1].sentence_idx)[:1000]
interact(see_sentences,
         sentence_idx=widgets.Dropdown(options=opt, 
         value=opt[0]))

interactive(children=(Dropdown(description='sentence_idx', options=('000_007', '000_011', '000_017', '000_020'…

<function __main__.see_sentences(sentence_idx)>

In [46]:
import qgrid
#most popular last-first
q = result_agg.groupby(['last_word','first_word']).agg({'sentence_idx':len, 'wrong_flag':np.mean})\
    .sort_values('sentence_idx',ascending=False)
qgrid.show_grid(q,show_toolbar=True)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [47]:
most_wrong = q[q.sentence_idx>=5].reset_index()
most_wrong.last_word.unique()

array(['ได้', 'ครับ', 'นี้', 'เลย', 'มาก', 'นั้น', 'ๆ', 'เรา', 'ผม', 'ไป',
       'แล้ว', 'ด้วย', 'อยู่', 'อย่างไร', '"', 'มัน', 'คุณ', 'มา', 'กัน',
       'ปี', 'ขึ้น', 'คน', 'ว่า', 'ค่ะ', 'ใหม่', 'เอง', 'ลง', 'อีก',
       'โลก', 'จริงๆ', 'ฉัน', 'เขา', 'นะ', 'เท่านั้น', 'เช่นกัน',
       'พวกเขา', 'อะไร', 'ไว้', 'ไหน', 'ต่างๆ', 'ที่สุด', 'เกิดขึ้น',
       'แบบนี้', 'ทำ', 'ดี', 'มากขึ้น', 'เหล่านั้น', 'มนุษย์', 'เดียว',
       'ก่อน', 'หนึ่ง', 'บ้าง', 'เหล่านี้', 'ทั่วไป', 'เห็น', 'ออกมา',
       'ตัวเอง', 'นะคะ', 'หมด', 'อื่นๆ', 'ธรรมชาติ', 'เมือง', 'ทุกคน',
       'ชีวิต', 'อย่าง', 'เข้ามา', 'ทีเดียว', 'ทั่วโลก', 'ผู้ชาย',
       'ที่นั่น', 'ทำได้', 'ครั้ง', 'มี', 'ตอนนี้', 'ตรงนี้', 'ปัจจุบัน',
       'ปลอดภัย', 'ก็ได้', 'จริง', 'เล่น', 'เหลือเกิน', 'กว่า',
       'เป็นไปได้', 'ดอลลาร์', 'มากมาย', 'รู้', 'เธอ', 'ดู', 'นิดหน่อย',
       'เดิม', 'มากกว่า', 'เปอร์เซ็นต์', 'นัก', 'ธุรกิจ', 'นั่นเอง',
       'อีกต่อไป', 'ข้อมูล', 'ถูกต้อง', 'ใช่', 'ทั้งหมด', 'แรก', 'ล่ะ',
       'เล็กน

In [48]:
most_wrong = q[q.sentence_idx>=5].reset_index()
most_wrong.first_word.unique()

array(['และ', 'ผม', 'แต่', 'เรา', 'มัน', 'ฉัน', 'ดังนั้น', 'นี่', 'คุณ',
       'ขอบคุณ', 'ใน', 'แล้ว', 'ค', '"', 'ถ้า', 'ไม่', 'เมื่อ', 'เขา',
       'พวกเขา', 'สิ่ง', 'มี', 'ลอง', 'ที่', 'ซึ่ง', 'การ', 'นั่น',
       'เพราะ', 'คน', 'ตอนนี้', 'ฉะนั้น', 'พวกเรา', 'เป็น', 'อย่าง',
       'พวก', 'ทีนี้', 'ด้วย', 'เรื่อง', 'โอเค', 'เอาล่ะ', 'สวัสดี',
       'ดิฉัน', 'เค', 'ตอนที่'], dtype=object)

In [49]:
#custom features
enders = ["ครับ","ค่ะ","คะ","นะคะ","นะ","จ้ะ","จ้า","จ๋า","ฮะ", #ending honorifics
          #enders
          "ๆ","ได้","แล้ว","ด้วย","เลย","มาก","กัน","เช่นกัน","เท่านั้น",
          "อยู่","ลง","ขึ้น","มา","ไป","ไว้","เอง","อีก","ใหม่","จริงๆ",
          "บ้าง","หมด","ทีเดียว","เดียว",
          #demonstratives
          "นั้น","นี้","เหล่านี้","เหล่านั้น",
          #questions
          "อย่างไร","หรือไม่","ไหน"]
starters = ["ผม","ฉัน","ดิฉัน","ชั้น","คุณ","มัน","เขา","เค้า",
            "เธอ","เรา","พวกเรา","พวกเขา", #pronouns
            #connectors
            "และ","หรือ","แต่","เมื่อ","ถ้า","ใน",
            "ด้วย","เพราะ","เนื่องจาก","ซึ่ง","ไม่",
            "ตอนนี้","ทีนี้","ดังนั้น","เพราะฉะนั้น","ฉะนั้น",
            #demonstratives
            "นั้น","นี้","เหล่านี้","เหล่านั้น"]

In [50]:
#most popular WRONG last-first
q = result_agg[result_agg.wrong_flag==1].groupby(['last_word','first_word'])\
    .agg({'sentence_idx':len, 'wrong_flag':np.mean})\
    .sort_values('sentence_idx',ascending=False)
qgrid.show_grid(q,show_toolbar=True)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [51]:
#most popular WRONG last
q = result_agg[result_agg.wrong_flag==1].groupby(['last_word'])\
    .agg({'sentence_idx':len, 'wrong_flag':np.mean})\
    .sort_values('sentence_idx',ascending=False)
qgrid.show_grid(q,show_toolbar=True)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [52]:
#most popular WRONG first
q = result_agg[result_agg.wrong_flag==1].groupby(['first_word'])\
    .agg({'sentence_idx':len, 'wrong_flag':np.mean})\
    .sort_values('sentence_idx',ascending=False)
qgrid.show_grid(q,show_toolbar=True)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…